# OncodriveCLUSTL

In [1]:
%matplotlib inline

import gzip
from collections import defaultdict

import numpy as np
import pandas as pd
from intervaltree import IntervalTree
from scipy.signal import argrelmax
import math as m

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from oncodcl_funct import *

### Automatically incorporates changes in the *oncodcl_funct* module 

In [2]:
%load_ext autoreload
%autoreload 2

#### Load parameters

In [3]:
# Smoothing
genemut = 2
smoothw = 50

# Clustering
clusterw = 50
clustermut = 2
clusterscore = 'nobias' # 'nmutations', 'genelenght', 'nobias'

### Parse regions


Why do some gene and transcript ids start with ENSGR or ENSTR in the GTF/GFF3?

The Ensembl ids, by convention, are made of a species index ("ENS" for human and "ENSMUS" for mouse) followed by a feature type indicator ("G" for gene, "T" for transcript, "E" for exon, "P" for translation) and an 11-number figure.

The GENCODE GTF/GFF3 files make an exception to this rule in the case of the so called "pseudoautosomal regions" (PAR) of chromosome Y. The gene annotation in these regions is identical between chromosomes X and Y. Ensembl do not provide different feature ids for both chromosomes. The Ensembl GTF file only includes this annotation once, for chromosome X. However, we decided that the GENCODE GTF/GFF3 files would include the annotation in the PAR regions of both chromosomes.

Since the GTF convention dictates that feature ids have to be unique for different genome regions, we slightly modify the Ensembl feature id by replacing the first zero with an "R". Thus, "ENSG00000182378.10" in chromosome X becomes "ENSGR0000182378.10" in chromosome Y.

Please note that this applies until release 24. From release 25, the PAR genes and transcripts have "_PAR_Y" appended to their ids.

This annotation is also labeled using the tag "PAR".

https://www.gencodegenes.org/faq.html


In [4]:
input_regions = '/home/carnedo/projects/oncodriveclustl/inputs/regions/02_cds.regions.gz'

In [5]:
%%time
trees, regions_d, chromosomes_d = regions(input_regions)

CPU times: user 12.6 s, sys: 52 ms, total: 12.7 s
Wall time: 12.7 s


### Read mutations file and intersect with regions

In [6]:
input_mutations = '/home/carnedo/projects/oncodriveclustl/inputs/mutations/pancanatlas/SKCM.txt'

In [7]:
%%time
mutations_d = read_mutations(input_mutations, trees)

CPU times: user 8.87 s, sys: 20 ms, total: 8.89 s
Wall time: 8.89 s


#### Testing problems 20170809

In [8]:
# plot clusters 

gene = 'GRIK3'

plot_max = np.zeros(len(total_regions[gene]['binary']))
plot_min_l = np.zeros(len(total_regions[gene]['binary']))
plot_min_r = np.zeros(len(total_regions[gene]['binary']))

# Change 0 to nan for matplotlib        
plot_max[ plot_max==0.0 ] = np.nan
plot_min_l[ plot_min_l==0.0 ] = np.nan 
plot_min_r[ plot_min_r==0.0 ] = np.nan


plt.figure(figsize=(26, 8))
ax1 = plt.subplot2grid((1, 5), (0, 0), colspan=4)

ax1.plot(binary, c='cornflowerblue')
ax1.set_ylabel('smoothing score', color='cornflowerblue', fontsize=20)

ax2 = ax1.twinx()
ax2.plot(mutations, c='red')
ax2.set_ylabel('n mutations', color='red', fontsize=20)

length = len(total_regions[gene]['binary'])

ax1.tick_params('y', colors='cornflowerblue', labelsize=15)
ax2.tick_params('y', colors='red', labelsize=15)

plt.title('TP53 Melanoma', fontsize=20)
#ax1.set_xlabel('Nucleotides', fontsize=20)
plt.savefig(plot)

NameError: name 'total_regions' is not defined

### Smooth

In [10]:
%%time
total_regions = defaultdict(dict)
for gene in regions_d.keys():
    # Analyze only genes with mutations >= n mutations
    if len(mutations_d[gene]) >= genemut:
        region_info = smoothing(regions=regions_d[gene], mutations=mutations_d[gene], window=smoothw)
        # dictionary with regions of all genes
        total_regions[region_info['symbol']] = region_info

len(total_regions.keys())

TypeError: smoothing() got an unexpected keyword argument 'regions'

## Clusters

- Find local maximum and minimum after smoothing for whole cds, according to smoothing score
- Define raw clusters: min(left), max, min(right) ----> Add filter? (ex. clusters wit max > smoothing score of 1 mutation)
- Starting from first maximum in the sequence, search for other relative maximum close to its min(right) border. 
- If there is one, check which of both maximum has a higher smoothing score and merge clusters by updating borders of the highest maximum. The lowest maximum (cluster) is removed from the dictionary of clusters. Iterate through all clusters until no updates are observed. 
- Filter clusters, minimum n mutations. Concerns: genes without clusters! 
- Score clusters. Two scores implemented: using position of smoothing maximum or mutation maximum within the cluster.

Questions: 
- Clusters calculation assuming + strand? YES ---> Take into account for plots

In [26]:
%%time
total_clusters = defaultdict(dict)

for gene in total_regions.keys():
    clusters = clustering(regions=total_regions[gene], mutations=mutations_d[gene], window=clusterw)
    if len(clusters.keys()) != 0: 
        total_clusters[gene] = clusters
    
print(len(total_clusters.keys()))

15561
CPU times: user 49.4 s, sys: 48 ms, total: 49.4 s
Wall time: 49.4 s


In [27]:
print(total_clusters['NRAS'])

defaultdict(<class 'dict'>, {0: {'score': 101.52699825132694, 'n_mutations': 117, 'min_r': [440, 0.0], 'max': [387, 4.1769383371525972], 'min_l': [359, 0.0]}, 1: {'score': 4.3284271247461898, 'n_mutations': 7, 'min_r': [562, 0.0], 'max': [534, 0.25070111049337263], 'min_l': [506, 0.0]}})


## Output results

### Read CGC genes

In [11]:
CGC = set([line.split('\t')[0] for line in open('/home/carnedo/projects/oncodriveclustl/inputs/CGC/CGCMay17_cancer_types_TCGA.tsv', 'r')])

### Tables

In [30]:
out = '/home/carnedo/projects/oncodriveclustl/outputs/'

In [31]:
#### Print table
clusters_table = out+'clusters_testing.tab'
with open(clusters_table, 'w') as fd:
    fd.write('Gene\tCluster\tMin_l\tMax\tMin_r\tWidth\tMut\tTotalmut\tScore\n')
    for gene, clusters in sorted(total_clusters.items()): 
        for cluster, values in clusters.items(): 
            if values['min_l'] != [] and values['min_r'] != []: 
                width = abs(values['min_l'][0] - values['min_r'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'][0], values['max'][0], values['min_r'][0], width, values['n_mutations'], len(mutations_d[gene]), values['score']))
            elif values['min_l'] == []: 
                width = abs(values['max'][0] - values['min_r'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'], values['max'][0], values['min_r'][0], width, values['n_mutations'], len(mutations_d[gene]), values['score']))
            elif values['min_r'] == []: 
                width = abs(values['min_l'][0] - values['max'][0])
                fd.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%.4f\n" %(gene, cluster, values['min_l'][0], values['max'][0], values['min_r'], width, values['n_mutations'], len(mutations_d[gene]), values['score']))

In [32]:
df = pd.read_csv(clusters_table, sep='\t', header=0)
df['CGC'] = df['Gene'].map(lambda x: x in CGC)
df.sort_values(by=['Score', 'CGC'], ascending=[False, False], inplace=True)
df.to_csv(path_or_buf=clusters_table, sep='\t', na_rep='')

In [33]:
df.head(n=15)

,Gene,Cluster,Min_l,Max,Min_r,Width,Mut,Totalmut,Score,CGC
9721,BRAF,2,495,523,608,113,247,269,228.4156,True
58335,NRAS,0,359,387,440,81,117,124,101.5270,True
39544,IDH1,4,827,854,880,53,17,23,16.4142,True
79748,SLC27A5,0,385,412,451,66,18,21,15.2537,False
49594,MAP2K1,1,327,371,414,87,20,28,15.0206,True
8998,BCL2L12,1,0,50,77,77,17,23,12.1217,False
71621,RAC1,1,15,86,125,110,18,20,11.8776,True
68820,PPP6C,1,80,125,166,86,15,29,11.0161,True
74822,RQCD1,1,368,394,420,52,10,13,10.0000,False
68212,POTEH,3,640,666,721,81,10,39,9.0000,False


In [34]:
df.to_csv(path_or_buf=clusters_table, sep='\t', na_rep='')

## Genes

In [41]:
genes_sum = defaultdict()

for gene, value in total_clusters.items(): 
    result = score_gene(clusters=value, cutoff = 2)
    genes_sum[gene] = result

In [43]:
genes_table = out+'testing.tab'

with open(genes_table, 'w') as fd:
    fd.write('Gene\tLength\tClusters\tMutations\tScore\n')
    for gene, result in genes_sum.items(): 
        fd.write("%s\t%s\t%s\t%s\t%s\n" %(gene, len(total_regions[gene]['genomic']), result[0], len(mutations_d[gene]), result[1]))
         
df = pd.read_csv(genes_table, sep='\t', header=0)
df['CGC'] = df['Gene'].map(lambda x: x in CGC)
df.sort_values(by=['Score', 'CGC'], ascending=[False, False], inplace=True)
df.to_csv(path_or_buf=genes_table, sep='\t', na_rep='')
df.head(n=15)

,Gene,Length,Clusters,Mutations,Score,CGC
5753,NRAS,571,2,124,52.927713,True
3132,BRAF,2337,5,269,46.739933,True
15136,RAC1,640,1,20,11.877612,True
8269,IDH1,1250,2,23,9.207107,True
1654,SLC27A5,2080,2,21,8.126845,False
6979,MAP2K1,1231,3,28,6.419666,True
8846,RQCD1,1046,2,13,5.801777,False
4744,BCL2L12,1038,3,23,5.227442,False
13306,AKR1B1,958,1,9,5.000000,False
12526,CCDC28A,828,1,13,4.339691,False


### Plot

In [ ]:
# plot clusters 

gene = 'TP53'
plot = out+'/figures/'+gene+'_'+str(genemut)+'_'+str(clustermut)+'_'+str(smoothw)+'_'+str(clusterw)+'_'+str(clusterscore)+'.png'

plot_max = np.zeros(len(total_regions[gene]['binary']))
plot_min_l = np.zeros(len(total_regions[gene]['binary']))
plot_min_r = np.zeros(len(total_regions[gene]['binary']))

for key, value in total_clusters[gene].items(): 
    plot_max[value['max'][0]]+=1
    plot_min_l[value['min_l'][0]]+=1
    plot_min_r[value['min_r'][0]]+=1

# Change 0 to nan for matplotlib        
plot_max[ plot_max==0.0 ] = np.nan
plot_min_l[ plot_min_l==0.0 ] = np.nan 
plot_min_r[ plot_min_r==0.0 ] = np.nan


if strands_d[gene] == '+':
    binary = total_regions[gene]['binary']
    mutations = total_regions[gene]['mutations']
    
elif strands_d[gene] == '-': 
    binary = total_regions[gene]['binary'][::-1]
    mutations = total_regions[gene]['mutations'][::-1]
    plot_max = plot_max[::-1]
    plot_min_l = plot_min_l[::-1]
    plot_min_r = plot_min_r[::-1]

plt.figure(figsize=(26, 8))
ax1 = plt.subplot2grid((1, 5), (0, 0), colspan=4)


ax1.plot(binary, c='cornflowerblue')
ax1.set_ylabel('smoothing score', color='cornflowerblue', fontsize=20)

ax2 = ax1.twinx()
ax2.plot(mutations, c='red')
ax2.set_ylabel('n mutations', color='red', fontsize=20)


ax1.plot(plot_max,'|', ms=20, c='green')
ax1.plot(plot_min_l,'|', ms=20, c='orange')
ax1.plot(plot_min_r,'|', ms=20, c='orange')

length = len(total_regions[gene]['binary'])

ax1.tick_params('y', colors='cornflowerblue', labelsize=15)
ax2.tick_params('y', colors='red', labelsize=15)

plt.title('TP53 Melanoma', fontsize=20)
#ax1.set_xlabel('Nucleotides', fontsize=20)
plt.savefig(plot)